3. For each xhtml  _corpus/raw_
   1. Get the element IDs associated with Item 7 and 7A
   2. TODO: Collect all the text inside [Item 7, Item 7A), save it to _corpus/text_

In [ ]:
from pathlib import Path
from lxml import etree
from tqdm.notebook import tqdm

In [ ]:
# def get_ref_ids(node: etree.Element) -> tuple[str, str]:
#     # you need to include the NS in every part of the `xpath`
#     # https://stackoverflow.com/questions/38936185/etree-xpath-return-entire-html-instead-of-text
#     ns_map = {'x':'http://www.w3.org/1999/xhtml'}
#     xpaths = [
#         ".//x:table/x:tr/x:td//*[contains(text(),'Financial Condition and Results of Operations')]",
#         "ancestor-or-self::x:a[contains(@href, '#')]",
#         "ancestor::x:tr/following-sibling::x:tr/x:td//x:a[contains(@href, '#')]"]
#     t1: etree.Element = node.xpath(xpaths[0], namespaces = ns_map)[0]
#     t1_href: str = t1.xpath(xpaths[1], namespaces = ns_map)[0].attrib['href']
#     t2_href: str = t1.xpath(xpaths[2], namespaces = ns_map)[0].attrib['href']
#     return t1_href, t2_href

In [ ]:
base_path = Path('data/10-K')

# Define your parser with the necessary settings
parser = etree.XMLParser(encoding='utf-8', recover=True, ns_clean=True)

def get_ref_ids(node: etree.Element) -> tuple[str, str]:
    ns_map = {'x': 'http://www.w3.org/1999/xhtml'}
    xpaths = [
        ".//x:table/x:tr/x:td//*[contains(text(),'Financial Condition and Results of Operations')]",
        "ancestor-or-self::x:a[contains(@href, '#')]",
        "ancestor::x:tr/following-sibling::x:tr/x:td//x:a[contains(@href, '#')]"
    ]
    # Attempt to find the element, but check if the result is empty
    t1_elements = node.xpath(xpaths[0], namespaces=ns_map)
    if not t1_elements:
        return ('', '')  # Return empty strings or some indication of not found

    t1 = t1_elements[0]
    t1_href_elements = t1.xpath(xpaths[1], namespaces=ns_map)
    t2_href_elements = t1.xpath(xpaths[2], namespaces=ns_map)

    # Check if the subsequent queries found anything
    t1_href = t1_href_elements[0].attrib['href'] if t1_href_elements else ''
    t2_href = t2_href_elements[0].attrib['href'] if t2_href_elements else ''

    return t1_href, t2_href


for year_folder in base_path.iterdir():
    if year_folder.is_dir():  # Ensure it's a directory
        for xhtml_file in year_folder.iterdir():
            if xhtml_file.suffix == '.xhtml':  # Process only .xhtml files
                print(f'Parsing {xhtml_file.name}...')
                with open(xhtml_file, mode='rb') as fp:
                    xhtml = fp.read()
                root: etree.Element = etree.fromstring(xhtml, parser)
                ids = get_ref_ids(root)
                print(ids[0])
                print(ids[1])

In [ ]:
# parser = etree.XMLParser(encoding = 'utf-8', recover = True, ns_clean = True)
# for xhtml_file in raw_files.iterdir():
#     print(f'Parsing {xhtml_file.name}...')
#     with open(xhtml_file, mode = 'rb') as fp:
#         xhtml = fp.read()
#     root: etree.Element = etree.fromstring(xhtml, parser)
#     ids = get_ref_ids(root)
#     print(ids[0])
#     print(ids[1])